In [12]:
import plotly.offline as py
from plotly.graph_objs import *
import pandas as pd
import math
import os
import numpy as np
py.init_notebook_mode()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


### Load File

In [3]:
cols=['px_est','py_est', 'v_est', 'vx_est', 'vy_est', 'yaw_est', 'yawrate_est', 'px_meas', 'py_meas', 'px_gt', 'py_gt', 'v_gt', 'vx_gt', 'vy_gt', 'yaw_gt', 'yawrate_gt', 'NIS_laser', 'NIS_radar']

In [5]:
with open('output.txt') as f:
    table_out = pd.read_table(f, sep='\t', header=None, names=cols, lineterminator='\n')

### Sample Data

In [9]:
print("Total data points: ", table_out.shape[0])
print("Each datapoint: ", table_out.shape[1])

Total data points:  499
Each datapoint:  18


In [11]:
table_out[485:500]

,px_est,py_est,v_est,vx_est,vy_est,yaw_est,yawrate_est,px_meas,py_meas,px_gt,py_gt,v_gt,vx_gt,vy_gt,yaw_gt,yawrate_gt,NIS_laser,NIS_radar
485,-10.54410,10.7713,5.17916,5.17434,-0.223422,-0.043152,-0.256054,-10.65630,10.8369,-10.61610,10.86090,5.187750,5.184350,0.187679,0.036185,-0.096263,2.45924,0.223296
486,-10.30170,10.7759,5.16395,5.15722,-0.263468,-0.051043,-0.252467,-10.35690,10.8697,5.18942,5.18687,0.162776,0.031372,-0.089450,1.153170,0.223296,NaN,NaN
487,-10.05360,10.8151,5.12986,5.12555,-0.210436,-0.041033,-0.228771,-9.90413,11.2255,-10.09760,10.87720,5.190970,5.189100,0.139618,0.026900,-0.082624,1.15317,8.258770
488,-9.79299,10.8575,5.14024,5.13667,-0.191410,-0.037246,-0.217378,-9.83809,10.8837,5.19241,5.19106,0.118212,0.022768,-0.075785,3.345360,8.258770,NaN,NaN
489,-9.55581,10.8442,5.11129,5.10571,-0.238732,-0.046724,-0.214493,-10.19500,10.3498,-9.57852,10.88910,5.193720,5.192780,0.098566,0.018979,-0.068933,3.34536,4.076960
490,-9.29468,10.8679,5.12106,5.11538,-0.241023,-0.047082,-0.206739,-9.31887,10.8936,5.19491,5.19428,0.080686,0.015532,-0.062071,1.514780,4.076960,NaN,NaN
491,-9.04611,10.8518,5.10324,5.09541,-0.282519,-0.055389,-0.202674,-9.22260,10.3527,-9.05915,10.89720,5.195970,5.195570,0.064579,0.012429,-0.055199,1.51478,2.868020
492,-8.78323,10.8786,5.11543,5.10806,-0.274571,-0.053701,-0.193842,-8.79937,10.9001,5.19691,5.19667,0.050248,0.009669,-0.048318,1.999650,2.868020,NaN,NaN
493,-8.53779,10.8442,5.09678,5.08537,-0.340764,-0.066909,-0.194702,-8.83914,10.0319,-8.53953,10.90230,5.197730,5.197590,0.037699,0.007253,-0.041430,1.99965,6.013900
494,-8.34373,10.7852,5.02901,5.00923,-0.445598,-0.088722,-0.203855,-8.27965,10.9039,5.19842,5.19835,0.026936,0.005182,-0.034535,5.095430,6.013900,NaN,NaN


### Helper Functions

In [13]:
def plot_nis(title, values, chi_95):
    
    t = list(range(0, len(values)))
    
    trace2 = Scatter(
        x=t,
        y=values,
        xaxis='x2',
        yaxis='y2',
        name = 'NIS',
        #mode = 'markers'
    )


    trace1= Scatter(
        x=[t[0], t[-1]],
        y=[chi_95, chi_95],
        xaxis='x2',
        yaxis='y2',
        name = '95 %',
        #mode = 'markers'
    )

    data = [trace1, trace2]

    layout = Layout(
        title=title,
        xaxis2=dict(
            anchor='x2',
            title='k'
        ),
        yaxis2=dict(
            anchor='y2',
            title='NIS'
        )
    )
    
    v = np.array(values)
    v_n = np.where(values > chi_95)[0]
    
    print(len(v_n) / len(v))

    fig = Figure(data=data, layout=layout)
    py.iplot(fig, filename= 'UKF')

NIS - Laser

In [14]:
plot_nis("NIS (Laser)", table_out['NIS_laser'], 5.991)

ValueError: 
    Invalid value of type 'pandas.core.series.Series' received for the 'y' property of scatter
        Received value: 0            NaN
1       0.000000
2            NaN
3       0.096093
4            NaN
5       0.220051
6            NaN
7       0.236887
8            NaN
9       0.004167
10           NaN
11      1.079240
12           NaN
13      2.173800
14           NaN
15      0.307758
16           NaN
17     12.489800
18           NaN
19      2.465390
20           NaN
21      1.548790
22           NaN
23      3.833780
24           NaN
25      0.345267
26           NaN
27      3.453880
28           NaN
29      0.606420
         ...    
469     1.145010
470          NaN
471     0.993627
472          NaN
473    11.406000
474          NaN
475     0.076796
476          NaN
477     6.541680
478          NaN
479     0.475934
480          NaN
481     3.904540
482          NaN
483     1.115180
484          NaN
485     2.459240
486          NaN
487     1.153170
488          NaN
4